# CODA + Elastic Transfer (PyTorch AE Demo)

This notebook mirrors the Model Lab flow with a small PyTorch AutoEncoder:
- CODA: applies per-step learning-rate scaling and per-sample weights
- Elastic Transfer: LoRA-style low-rank delta between trials (function-preserving warm start)
- Energy Ledger: records collisions/allocations for evidence


In [ ]:
import os, json, math, random, time
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, WeightedRandomSampler

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device:', device)

# Synthetic dataset (similar to claims features)
N = 20000
rng = np.random.default_rng(42)
submitted = rng.normal(800, 150, N)
allowed   = submitted * rng.uniform(0.6, 0.9, N)
paid      = allowed * rng.uniform(0.6, 0.95, N)
lag       = rng.integers(0, 20, N)
X = np.stack([submitted/1000, allowed/1000, paid/1000, lag/20], axis=1).astype(np.float32)
X_t = torch.tensor(X)

# AE model
class AE(nn.Module):
    def __init__(self, d=4, h1=16, z=3, h2=16):
        super().__init__()
        self.enc = nn.Sequential(nn.Linear(d, h1), nn.ReLU(), nn.Linear(h1, z), nn.ReLU())
        self.dec = nn.Sequential(nn.Linear(z, h2), nn.ReLU(), nn.Linear(h2, d))
    def forward(self, x):
        z = self.enc(x)
        return self.dec(z)

mse = nn.MSELoss(reduction='none')

# CODA signals (demo)
def estimate_mass(losses):
    v = losses.var().item() if isinstance(losses, torch.Tensor) else np.var(losses)
    base = 1.0 / (math.sqrt(v) + 1e-6)
    mean = losses.mean().item() if isinstance(losses, torch.Tensor) else float(np.mean(losses))
    if isinstance(losses, torch.Tensor): losses = losses.detach().cpu().numpy()
    return base / (np.abs(losses - mean) + 1e-6)

def coda_step(window_energy, mass, info_gain):
    target_v = info_gain / (mass + 1e-6)
    scale = window_energy / (target_v.sum() + 1e-8)
    v = target_v * scale
    lr_scale = v / (np.mean(v) + 1e-9)
    sample_weight = info_gain / (info_gain.max() + 1e-9)
    return lr_scale.astype(np.float32), sample_weight.astype(np.float32)

# Elastic transfer (LoRA-like delta)
def lora_delta(W, rank=2, scale=0.01):
    A = torch.randn(W.shape[0], rank, device=W.device) * scale
    B = torch.randn(rank, W.shape[1], device=W.device) * scale
    return A @ B

def apply_lora(model, rank=2, scale=0.01):
    with torch.no_grad():
        for mod in model.modules():
            if isinstance(mod, nn.Linear):
                mod.weight += lora_delta(mod.weight, rank, scale)

# Energy ledger
ledger = { 'entries': [], 'windowEnergy': 1.0 }

def log(entry):
    entry['time'] = time.strftime('%H:%M:%S')
    ledger['entries'].append(entry)

# Training loop with CODA + Elastic Transfer
model = AE().to(device)
opt = optim.Adam(model.parameters(), lr=1e-3)

steps = 300
batch_size = 128

for t in range(3):  # 3 trials to show collisions
    # Elastic collision: warm start next trial with LoRA delta
    if t > 0:
        apply_lora(model, rank=4, scale=0.005)
        log({'type':'collision', 'from': f'trial{t-1}', 'to': f'trial{t}', 'deltaEnergy': 0})

    # compute initial per-sample losses for CODA mass/IG
    with torch.no_grad():
        out0 = model(X_t.to(device))
        losses0 = mse(out0, X_t.to(device)).mean(dim=1).cpu().numpy()
    mass = estimate_mass(losses0)
    info_gain = np.maximum(1e-3, losses0)
    lr_scale, sample_weight = coda_step(1.0, mass, info_gain)

    sampler = WeightedRandomSampler(sample_weight, num_samples=len(sample_weight), replacement=True)
    dl = DataLoader(TensorDataset(X_t), batch_size=batch_size, sampler=sampler)

    for step, (xb,) in enumerate(dl):
        if step >= steps: break
        scale = float(lr_scale[step % len(lr_scale)])
        for g in opt.param_groups:
            g['lr'] = 1e-3 * scale
        xb = xb.to(device)
        opt.zero_grad()
        xhat = model(xb)
        loss = mse(xhat, xb).mean()
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        opt.step()
        if step % 50 == 0:
            log({'type':'allocation', 'deltaEnergy': 1.0, 'details': {'lrScale': round(scale,3), 'trial': t, 'step': step, 'loss': float(loss.item())}})

print('Done. Ledger entries:', len(ledger['entries']))



In [ ]:
# Save ledger to JSON (local or DBFS if mounted)
import json, os
path = os.getenv('LEDGER_JSON_PATH', 'energy_ledger.json')
with open(path, 'w') as f:
    json.dump(ledger, f, indent=2)
print('Saved ledger to', path)


In [ ]:
# Optional: write ledger to Delta if Spark is available
try:
    spark  # type: ignore
    from pyspark.sql import Row
    rows = []
    for e in ledger['entries']:
        rows.append(Row(time=e.get('time'), type=e.get('type'), from_=e.get('from'), to=e.get('to'), deltaEnergy=float(e.get('deltaEnergy', 0.0)), details=json.dumps(e.get('details', {}))))
    df = spark.createDataFrame(rows)
    spark.sql("CREATE DATABASE IF NOT EXISTS aethergen")
    df.write.mode('overwrite').format('delta').saveAsTable('aethergen.energy_ledger')
    print('Delta table written: aethergen.energy_ledger')
except NameError:
    print('Spark not available; skipped Delta write')
